In [1]:
import pandas as pd
import requests

In [2]:
API_KEY = '881078-ec5446d8-7fbd-4fac-806d-8a4d81eece36'

In [103]:
next_url = "https://api.agendor.com.br/v3/deals"

# Cabeçalhos da requisição
headers = {
    "Authorization": f"Token {API_KEY}",
    "Content-Type": "application/json"
}

params={"per_page": 100}

all_json_deals_data = []

while next_url:
    response = requests.get(next_url, headers=headers, params=params)
    json_deal_data = response.json()['data']
    all_json_deals_data.extend(json_deal_data)
    links = response.json()['links']
    if 'next' in links.keys():
        next_url = links['next']
    else:
        next_url = False




In [111]:
import json

# Salvando como JSON
with open("deals.json", "w", encoding="utf-8") as arquivo:
    json.dump(all_json_deals_data, arquivo, indent=4, ensure_ascii=False)

In [113]:
import json

# Ler arquivo JSON com vários objetos
with open("deals.json", "r", encoding="utf-8") as arquivo:
    deals = json.load(arquivo)

In [115]:
deals

[{'id': 29347766,
  'accountId': 698720,
  'organization': None,
  'person': {'id': 57602860, 'name': 'Victoria', 'email': None},
  'author': {'accountId': 698720, 'id': 863021, 'name': 'Juliane Sales'},
  'dealStage': {'id': 3261930,
   'name': '1.5 CONTRATO',
   'sequence': 5,
   'funnel': {'id': 772312, 'name': '1 POS'}},
  'dealStatus': {'id': 1, 'name': 'Em andamento'},
  'lossReason': None,
  'owner': {'accountId': 698720, 'id': 863021, 'name': 'Juliane Sales'},
  'title': 'Victoria Porto Carvalho',
  'description': None,
  'startTime': '2025-02-04T18:13:24.000Z',
  'endTime': None,
  'wonAt': None,
  'lostAt': None,
  'createdAt': '2025-02-04T18:13:24.000Z',
  'updatedAt': '2025-02-04T18:13:38.000Z',
  'ranking': 0,
  'value': 0.0,
  'products': [],
  'allowedUsers': [{'accountId': 698720,
    'id': 860639,
    'name': 'Guilherme Hely'},
   {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'},
   {'accountId': 698720, 'id': 860980, 'name': 'Gustavo Porto'},
   {'accountId

In [14]:
deals

[{'id': 29334556,
  'accountId': 698720,
  'organization': None,
  'person': {'id': 57593599, 'name': 'Márcio Nunes Chagas', 'email': None},
  'author': {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'},
  'dealStage': {'id': 3261930,
   'name': '1.5 CONTRATO',
   'sequence': 5,
   'funnel': {'id': 772312, 'name': '1 POS'}},
  'dealStatus': {'id': 2, 'name': 'Ganho'},
  'lossReason': None,
  'owner': {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'},
  'title': 'Márcio Nunes Chagas',
  'description': None,
  'startTime': '2025-02-04T14:06:20.000Z',
  'endTime': '2025-02-04T00:00:00.000Z',
  'wonAt': '2025-02-04T15:15:55.000Z',
  'lostAt': None,
  'createdAt': '2025-02-04T14:06:20.000Z',
  'updatedAt': '2025-02-04T15:15:55.000Z',
  'ranking': 0,
  'value': 0.0,
  'products': [],
  'allowedUsers': [{'accountId': 698720,
    'id': 860639,
    'name': 'Guilherme Hely'},
   {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'},
   {'accountId': 698720, 'id': 860980